In [ ]:
!pip install transformers[torch]
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from sklearn.metrics import classification_report
from datasets import Dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification
import pandas as pd
import numpy as np


In [ ]:
import spacy
!python -m spacy download es_core_news_sm

# Cargar el modelo de spaCy para tokenizar oraciones
nlp = spacy.load("es_core_news_sm")


In [ ]:
# Función para obtener los enunciados con las keywords
def extract_keyword_sentences(row):
    keywords = set(row['keyword'].split(', '))  # Convertir las keywords en un conjunto para eliminar duplicados
    text = row['text']
    doc = nlp(text)
    found_sentences = set()  # Conjunto para almacenar las oraciones encontradas
    for sentence in doc.sents:
        # Verificar si la oración contiene al menos una de las keywords
        if any(keyword.lower() in sentence.text.lower() for keyword in keywords):
            found_sentences.add(sentence.text.strip())  # Agregar la oración al conjunto de oraciones encontradas
    return '. '.join(found_sentences) if found_sentences else None  # Unir las oraciones encontradas en un solo texto

In [ ]:
# Cargar los datos de entrenamiento
data = pd.read_csv('SMM4H_2024_Task3_Training_1800.csv', nrows=1800, usecols=[0, 1, 2, 3], engine='python')
# Filtrar el DataFrame para quedarse solo con las filas donde 'label' es diferente de 0
data_filtered = data.loc[data['label'] != 0]


In [ ]:
# Cargar los datos de VALIDACIÓN
data_val = pd.read_csv('SMM4H_2024_Task3_Validation_600_codalab.csv', nrows=600, usecols=[0, 1, 2, 3], engine='python')
# Filtrar el DataFrame para quedarse solo con las filas donde 'label' es diferente de 0
data_val_filtered = data_val.loc[data_val['label'] != 0]


In [ ]:
print(data_filtered)
print(data_val_filtered)


           id                                keyword  \
1131  d3moq94                                   walk   
1132  d51rpnb                          outside, bike   
1133  d5bzg04                                   walk   
1134  d5ttkj7                                    run   
1135  d6se5to              running, run, horse, walk   
...       ...                                    ...   
1795   gqzye9                     pool, beach,  pool   
1796   env299                     outside , outdoors   
1797  e9bnr1s                                Jogging   
1798   qrmhbe                  walk, swimming,  pool   
1799   mxbsm8  roller blade, outside , roller blades   

                                                   text  label  
1131   Do you feel like the texts that you send back...      1  
1132   I'm gonna do the Pokemon thing to get myself ...      1  
1133   Something that work for me is to expose mysel...      1  
1134  Absolutely! Please encourage your son to do so...      1  
11

In [ ]:
# Cargar los datos aumentados
data_aug_c1_1A1 = pd.read_csv('augmented_data_CLASE_1_1A1_200_palabras.csv', engine='python')
data_aug_c3_1A1 = pd.read_csv('augmented_data_CLASE_3_1A1_370_palabras.csv', engine='python')
data_aug_c1 = pd.read_csv('augmented_data_CLASE_1_2A1_200_palabras.csv', engine='python')
data_aug_c3 = pd.read_csv('augmented_data_CLASE_3_2A1_370_palabras.csv', engine='python')


In [ ]:
# Crear un nuevo DataFrame con las columnas 'text' y 'label' del DataFrame original
new_dataframe = pd.DataFrame({'text': data_filtered['text'].copy(), 'label': data_filtered['label'].copy()})
print(new_dataframe)

                                                   text  label
1131   Do you feel like the texts that you send back...      1
1132   I'm gonna do the Pokemon thing to get myself ...      1
1133   Something that work for me is to expose mysel...      1
1134  Absolutely! Please encourage your son to do so...      1
1135  I so agree its such a feel good thing for me. ...      1
...                                                 ...    ...
1795   Social Anxiety: The Essentials. After looking...      3
1796   Eye contact. Being in public. I can't do it. ...      3
1797  If you look for the light, you will find it. I...      3
1798   I feel like my SA is ruining my life. Hi, I h...      3
1799  I got social anxiety trying to roller blade. S...      3

[669 rows x 2 columns]


In [ ]:
# Concatenar los DataFrames 'new_dataframe', 'data_aug_c1' y 'data_aug_c3'
concatenated_data = pd.concat([new_dataframe, data_aug_c1_1A1, data_aug_c3_1A1, data_aug_c1, data_aug_c3], ignore_index=True)
# Filtrar las filas con valores de 'label' igual a 1, 2 o 3
concatened_data = concatenated_data.loc[concatenated_data['label'].isin([1, 2, 3])]

print(concatenated_data)

                                                   text  label
0      Do you feel like the texts that you send back...      1
1      I'm gonna do the Pokemon thing to get myself ...      1
2      Something that work for me is to expose mysel...      1
3     Absolutely! Please encourage your son to do so...      1
4     I so agree its such a feel good thing for me. ...      1
...                                                 ...    ...
1212  I am a 24-year-old college grad who has spent ...      3
1213  I'm a 17-year-old who used to be quiet and int...      3
1214  Despite my efforts to recover my SA, I still s...      3
1215  I'm 17F and I have Social Anxiety, as well as ...      3
1216  I have been longing for roller blades for some...      3

[1217 rows x 2 columns]


In [ ]:
 # Filtrar y actualizar el texto con los enunciados que contienen las keywords
#data_filtered['text'] = data_filtered.apply(lambda row: extract_keyword_sentences(row), axis=1)
# Convertir las etiquetas a un rango de 0 a 2
concatened_data['label'] -= 1
data_val_filtered['label'] -=1


<ipython-input-10-0315bffc643e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_val_filtered['label'] -=1


In [ ]:
print(data_val_filtered['label'].values)
print(concatened_data['label'].values)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]
[0 0 0 ... 2 2 2]


In [ ]:
# Estructurar los datos en formato de diccionario
train_data = {
    'text': concatened_data['text'].tolist(),
    'label': concatened_data['label'].tolist()
}

val_data = {
    'text': data_val_filtered['text'].tolist(),
    'label': data_val_filtered['label'].tolist()
}

In [ ]:
# Cargar el tokenizador y el modelo
tokenizer = AutoTokenizer.from_pretrained('mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis')
model = AutoModelForSequenceClassification.from_pretrained('mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

In [ ]:
# Función para realizar el preprocesamiento de los datos
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=512, padding=True)


In [ ]:
# Convertir los datos en objetos Dataset
train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)

# Aplicar el preprocesamiento a los datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1217 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

In [ ]:
# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)


In [ ]:
# Función para calcular las métricas
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    accuracy = (predictions == labels).mean()
    report = classification_report(labels, predictions)
    print(report)
    return {"accuracy": accuracy}

In [ ]:
# Iniciar el entrenamiento
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Entrenar el modelo
trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.041057,0.560538
2,No log,1.000542,0.591928
3,No log,1.068044,0.573991
4,0.696200,1.120608,0.569507


              precision    recall  f1-score   support

           0       0.41      0.48      0.44        54
           1       0.74      0.63      0.68       131
           2       0.33      0.42      0.37        38

    accuracy                           0.56       223
   macro avg       0.50      0.51      0.50       223
weighted avg       0.59      0.56      0.57       223

              precision    recall  f1-score   support

           0       0.43      0.41      0.42        54
           1       0.70      0.73      0.71       131
           2       0.41      0.39      0.40        38

    accuracy                           0.59       223
   macro avg       0.51      0.51      0.51       223
weighted avg       0.59      0.59      0.59       223

              precision    recall  f1-score   support

           0       0.41      0.39      0.40        54
           1       0.71      0.66      0.69       131
           2       0.40      0.53      0.45        38

    accuracy        

TrainOutput(global_step=612, training_loss=0.6397791345135059, metrics={'train_runtime': 289.4469, 'train_samples_per_second': 16.818, 'train_steps_per_second': 2.114, 'total_flos': 644862796664832.0, 'train_loss': 0.6397791345135059, 'epoch': 4.0})

In [ ]:
import shutil
from google.colab import drive

# Monta Google Drive
drive.mount('/content/drive')

# Define la ruta de la carpeta que deseas guardar en Google Drive
carpeta_colab = '/content/results'  # Cambia esto por la ruta de tu carpeta en Colab
carpeta_drive = '/content/drive/MyDrive/FINE_SENTIMENT_6_data_aug_1A1_2A1_c1_c3'  # Cambia esto por la ruta donde deseas guardar la carpeta en Drive

# Copia la carpeta de Colab a Drive
shutil.copytree(carpeta_colab, carpeta_drive)

Mounted at /content/drive


'/content/drive/MyDrive/FINE_SENTIMENT_6_data_aug_1A1_2A1_c1_c3'

### **PRUEBA DE MODELO REEENTRENADO CON DATOS DE VALIDACIÓN CODALAB**

In [ ]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification, pipeline

# Ruta del modelo entrenado en Google Drive
modelo_entrenado_path = '/content/results/checkpoint-306'

# Cargar el tokenizador y el modelo
tokenizer = AutoTokenizer.from_pretrained('mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis')
model = AutoModelForSequenceClassification.from_pretrained(modelo_entrenado_path)

# Crear el clasificador de sentimiento
sentiment_task = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, truncation=True, max_length=512)


In [ ]:
# Cargar los datos de validación
datos_validacion = pd.read_csv('SMM4H_2024_Task3_Validation_600_codalab.csv', usecols=[0,1,2,3])

# Filtrar el DataFrame para quedarse solo con las filas donde 'label' es diferente de 0
datos_validacion = datos_validacion.loc[datos_validacion['label'] != 0]


In [ ]:
print(datos_validacion)

          id                                   keyword  \
377   3y7rva                                   running   
378   4evl7n                                camp, hike   
379   4gpbsy                                       run   
380   4vmwlg  running, run, runs, walk, horse, jogging   
381   5031vh                                   outside   
..       ...                                       ...   
595  edvs552                                      walk   
596   ee31pf                                   outside   
597  eei3pz3                             outside, walk   
598  eek8bpk                                   outside   
599  eeljxq0                                   outside   

                                                  text  label  
377  Self worth and anxiety. Hi guys. 25 (M) I am n...      1  
378  Pushing Myself Massively. I'm going on an over...      1  
379  You are not your inner critic.. Hey everyone. ...      1  
380  Just Started Running - A Question to My So

In [ ]:
# Aplicar el clasificador de sentimiento a los textos del archivo
resultados = []
for texto in datos_validacion['text']:
    output = sentiment_task(texto)
    resultados.append(output)

# Imprimir los resultados
for i, resultado in enumerate(resultados):
    print(f"Texto {i+1}: {resultado}")

Texto 1: [{'label': 'neutral', 'score': 0.457002192735672}]
Texto 2: [{'label': 'neutral', 'score': 0.8544274568557739}]
Texto 3: [{'label': 'negative', 'score': 0.5124530792236328}]
Texto 4: [{'label': 'neutral', 'score': 0.9219464063644409}]
Texto 5: [{'label': 'negative', 'score': 0.7974788546562195}]
Texto 6: [{'label': 'positive', 'score': 0.5990985631942749}]
Texto 7: [{'label': 'negative', 'score': 0.6375998258590698}]
Texto 8: [{'label': 'negative', 'score': 0.7054887413978577}]
Texto 9: [{'label': 'negative', 'score': 0.9795659184455872}]
Texto 10: [{'label': 'neutral', 'score': 0.6928262114524841}]
Texto 11: [{'label': 'negative', 'score': 0.6068462133407593}]
Texto 12: [{'label': 'neutral', 'score': 0.8785402178764343}]
Texto 13: [{'label': 'neutral', 'score': 0.8718218803405762}]
Texto 14: [{'label': 'negative', 'score': 0.9880499243736267}]
Texto 15: [{'label': 'neutral', 'score': 0.8021873831748962}]
Texto 16: [{'label': 'neutral', 'score': 0.8305076360702515}]
Texto 17: 

In [ ]:
print(resultados)

[[{'label': 'neutral', 'score': 0.457002192735672}], [{'label': 'neutral', 'score': 0.8544274568557739}], [{'label': 'negative', 'score': 0.5124530792236328}], [{'label': 'neutral', 'score': 0.9219464063644409}], [{'label': 'negative', 'score': 0.7974788546562195}], [{'label': 'positive', 'score': 0.5990985631942749}], [{'label': 'negative', 'score': 0.6375998258590698}], [{'label': 'negative', 'score': 0.7054887413978577}], [{'label': 'negative', 'score': 0.9795659184455872}], [{'label': 'neutral', 'score': 0.6928262114524841}], [{'label': 'negative', 'score': 0.6068462133407593}], [{'label': 'neutral', 'score': 0.8785402178764343}], [{'label': 'neutral', 'score': 0.8718218803405762}], [{'label': 'negative', 'score': 0.9880499243736267}], [{'label': 'neutral', 'score': 0.8021873831748962}], [{'label': 'neutral', 'score': 0.8305076360702515}], [{'label': 'positive', 'score': 0.42271697521209717}], [{'label': 'neutral', 'score': 0.4607725739479065}], [{'label': 'positive', 'score': 0.58

In [ ]:
# Función para mapear las etiquetas
def map_label(label):
    if label == 'positive':
        return 1
    elif label == 'neutral':
        return 2
    elif label == 'negative':
        return 3
    else:
        return None

In [ ]:
# Mapear las etiquetas de los resultados
mapeados = []
for resultado in resultados:
    etiqueta_original = resultado[0]['label']
    etiqueta_mapeada = map_label(etiqueta_original)
    mapeados.append(etiqueta_mapeada)

In [ ]:
print(mapeados)

[2, 2, 3, 2, 3, 1, 3, 3, 3, 2, 3, 2, 2, 3, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 1, 1, 2, 3, 1, 2, 3, 2, 2, 2, 2, 3, 2, 3, 3, 2, 1, 1, 2, 3, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 2, 3, 3, 2, 1, 2, 2, 2, 3, 1, 2, 3, 3, 1, 1, 1, 2, 1, 1, 1, 3, 3, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 1, 3, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2, 3, 3, 2, 1, 2, 1, 2, 1, 1, 2, 2, 3, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 1, 3, 3, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 1, 2, 2, 3, 2, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 3]


In [ ]:
y_true = datos_validacion['label']
y_pred = mapeados

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           1       0.22      0.15      0.18        54
           2       0.70      0.73      0.71       131
           3       0.14      0.18      0.16        38

    accuracy                           0.49       223
   macro avg       0.35      0.35      0.35       223
weighted avg       0.49      0.49      0.49       223

